# Requirement:
- numpy 1.19.5
- tensorflow 2.5.0
- tensorflow_privacy 0.6.1
- sklearn 0.24.2
- xgboost 1.4.2

Before performing the experiments, you need to fit the **mia_path** (path for MIA package) and **pickle_file**(path for tabular QMNIST data) according to your own PC.

In [1]:
mia_path = './MIA'

import sys
sys.path.append(mia_path)

import numpy as np

import pickle
import tensorflow as tf
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

from mia.estimators import ShadowModelBundle, AttackModelBundle, prepare_attack_data

# Load QMNIST data.

In [3]:
pickle_file = '/home/jiangnan/Desktop/dataset/QMNIST_tabular.pickle'

with open(pickle_file, 'rb') as f:
  pickle_data = pickle.load(f)
  x_defender = pickle_data['x_private']
  x_reserve = pickle_data['x_reserved']
  y_defender = pickle_data['y_private']
  y_reserve = pickle_data['y_reserved']
  del pickle_data
print('Data loaded.')

Data loaded.


In [4]:
NUM_CLASSES = 10

y_defender = y_defender[:,0]
y_reserve = y_reserve[:,0]

y_defender = np.expand_dims(y_defender,axis=1)
y_reserve = np.expand_dims(y_reserve,axis=1)

y_defender = tf.keras.utils.to_categorical(y_defender, num_classes=NUM_CLASSES)
y_reserve = tf.keras.utils.to_categorical(y_reserve, num_classes=NUM_CLASSES)

# Defender model $M_D$

In [5]:
#l2_norm_clip = 1.0
#noise_multiplier = 1.1

def defender_model_fn():
    """The architecture of the defender (victim) model.
    The attack is white-box, hence the attacker is assumed to know this architecture too."""

    model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation=tf.nn.relu),
    tf.keras.layers.Dense(10, activation=tf.nn.softmax)
    ])
    
    ##if train_with_DP:
    ##from tensorflow_privacy.privacy.optimizers.dp_optimizer_keras import DPKerasSGDOptimizer
    ##train_op = DPKerasSGDOptimizer(
    ##    l2_norm_clip=l2_norm_clip,
    ##    noise_multiplier=noise_multiplier,
    ##    num_microbatches=1, # Possible problem after reducing the size of cost vector in tensorflow-privacy. Check: https://github.com/tensorflow/privacy/issues/17
    ##    learning_rate=1e-4
    ##    )
    ##else:
    
    train_op = tf.optimizers.Adam(1e-4)
    
    model.compile(optimizer=train_op,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model


## Train $M_D$ (For this version there is no "mock attacker")

In [28]:
number_records = 10000

# Train the defender model.
print("Training the defender model...")
defender_model = defender_model_fn()
defender_model.fit(
    x_defender[:number_records], y_defender[:number_records], epochs=30, batch_size = 32, validation_split=0.5, verbose=True
)

Training the defender model...
Epoch 1/30
157/157 [==============================] - 1s 3ms/step - loss: 5.1662 - accuracy: 0.3930 - val_loss: 1.9022 - val_accuracy: 0.6062
Epoch 2/30
157/157 [==============================] - 0s 3ms/step - loss: 1.2524 - accuracy: 0.7156 - val_loss: 1.0761 - val_accuracy: 0.7422
Epoch 3/30
157/157 [==============================] - 0s 3ms/step - loss: 0.7783 - accuracy: 0.8008 - val_loss: 0.7831 - val_accuracy: 0.8036
Epoch 4/30
157/157 [==============================] - 0s 3ms/step - loss: 0.5704 - accuracy: 0.8498 - val_loss: 0.6611 - val_accuracy: 0.8346
Epoch 5/30
157/157 [==============================] - 0s 2ms/step - loss: 0.4496 - accuracy: 0.8802 - val_loss: 0.5769 - val_accuracy: 0.8588
Epoch 6/30
157/157 [==============================] - 0s 3ms/step - loss: 0.3692 - accuracy: 0.8984 - val_loss: 0.5212 - val_accuracy: 0.8658
Epoch 7/30
157/157 [==============================] - 0s 2ms/step - loss: 0.3059 - accuracy: 0.9146 - val_loss: 0.475

In [7]:
defender_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               65664     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1290      
Total params: 66,954
Trainable params: 66,954
Non-trainable params: 0
_________________________________________________________________


## Evaluate the utility of $M_D$ on utility evaluation dataset $E_U$ (which is equal to reserve dataset $R$).

Evaluation metrics: **Accuracy** & **AUC**

In [8]:
from sklearn import metrics
from sklearn.metrics import log_loss
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances

In [9]:
# compute the accuracy as acc
predict_reserve_proba = defender_model.predict_proba(x_reserve)
predict_reserve = np.argmax(predict_reserve_proba, axis=1)
label_reserve = np.argmax(y_reserve, axis=1)
acc = accuracy_score(label_reserve, predict_reserve)

/home/jiangnan/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [10]:
auc_by_class = []

# compute auc per class then take the average value
for i in range(NUM_CLASSES):
  class_indices = np.argmax(y_reserve, axis=1) == i
  fpr, tpr, thresholds = metrics.roc_curve(class_indices, predict_reserve_proba[:,i])
  auc = metrics.auc(fpr, tpr)
  auc_by_class.append(auc)

average_auc = np.mean(auc_by_class)

In [11]:
print('Utility of defender model:')
print('Acc: {}'.format(acc))
print('Auc: {}'.format(average_auc))

Utility of defender model:
Acc: 0.9347022332506203
Auc: 0.9963099672395271


In [17]:
# if necessary, save or reload the defender model trained

#defender_model_path = '/home/jiangnan/Desktop/model/QMNIST_defender_model'
#defender_model.save(defender_model_path)

#defender_model = tf.keras.models.load_model(defender_model_path)

# Attack model $M_A$

In [29]:
# Test the success of the attack.

# Prepare examples that were in the training, and out of the training.
data_in = x_defender[:number_records], y_defender[:number_records]
data_out = x_reserve[:number_records], y_reserve[:number_records]

# Compile them into the expected format for the AttackModelBundle.
attack_test_data, attack_test_data_dense2, real_membership_labels = prepare_attack_data(
    defender_model, data_in, data_out
)


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 512)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_3 (Dense)              (None, 10)                1290      
Total params: 66,954
Trainable params: 66,954
Non-trainable params: 0
_________________________________________________________________
None


/home/jiangnan/.local/lib/python3.8/site-packages/tensorflow/python/keras/engine/sequential.py:430: UserWarning: `model.predict_proba()` is deprecated and will be removed after 2021-01-01. Please use `model.predict()` instead.
  warnings.warn('`model.predict_proba()` is deprecated and '


In [30]:
print(attack_test_data.shape)
print(attack_test_data_dense2.shape)
print(real_membership_labels.shape)

(20000, 20)
(20000, 138)
(20000,)


In [31]:
import random
import xgboost as xgb
from tqdm import tqdm

In [32]:
# compute euclidean distance(ed) and cross_entropy(ce) loss for each data sample
ed = np.array([])
ce = np.array([])

for i in range(attack_test_data.shape[0]):
  ed = np.hstack((ed,euclidean_distances(attack_test_data[i,NUM_CLASSES:].reshape((1,NUM_CLASSES)), attack_test_data[i,:NUM_CLASSES].reshape((1,NUM_CLASSES))).squeeze()))
  ce = np.hstack((ce,log_loss(attack_test_data[i,NUM_CLASSES:].reshape((1,NUM_CLASSES)), attack_test_data[i,:NUM_CLASSES].reshape((1,NUM_CLASSES)))))

ed = ed.reshape((len(ed),1))
ce = ce.reshape((len(ce),1))


## Evaluate the dumb Attacker

In [33]:
fpr, tpr, thresholds = metrics.roc_curve(real_membership_labels, ce)
auc = metrics.auc(fpr, tpr)
print('Dumb Attacker AUC: {}'.format(auc))

Dumb Attacker AUC: 0.495148485


## Train & Evaluate the oracle Attacker (Leave two out)

In [34]:
# combine the attack_test_data(input of the oracle attack model) with their cross_entropy loss and euclidean distance
# but this seems won't help us to get a better attack performance.
new_attack_test_data = np.hstack((attack_test_data,ce,ed))
new_attack_test_data.shape

(20000, 22)

In [62]:
complete_attack_test_data = np.hstack((attack_test_data_dense2[:,:-10], new_attack_test_data))
complete_attack_test_data.shape

(20000, 150)

In [63]:
def compute_attack_model(attack_data, attack_labels, trials=1000):
  # repeat the evaluating process for 'trials' times.
  
  xgb_attack_guesses = np.array([])
  xgb_attack_proba = np.reshape(np.array([[],[]]),(0,2))
  xgb_attack_real = np.array([])


  for i in tqdm(range(trials)):

    random_index = random.randint(0,number_records)
    indexs = [random_index, random_index+number_records]

    evaluation_data = attack_data[indexs]
    evaluation_label = attack_labels[indexs]

    xgb_data = np.delete(attack_data, indexs, axis=0)
    xgb_label = np.delete(attack_labels, indexs, axis=0)

    xgb_model = xgb.XGBClassifier(objective="binary:logistic", random_state=50, eval_metric='logloss')
    xgb_model.fit(attack_data, attack_labels)

    xgb_guesses = xgb_model.predict(evaluation_data)
    xgb_proba = xgb_model.predict_proba(evaluation_data)

    xgb_attack_guesses = np.hstack((xgb_attack_guesses,xgb_guesses))
    xgb_attack_proba = np.vstack((xgb_attack_proba,xgb_proba))
    xgb_attack_real = np.hstack((xgb_attack_real,evaluation_label))

  return xgb_attack_guesses, xgb_attack_proba, xgb_attack_real

In [68]:
xgb_attack_guesses, xgb_attack_proba, xgb_attack_real = compute_attack_model(complete_attack_test_data, real_membership_labels,100)

  0%|          | 0/100 [00:00<?, ?it/s]/home/jiangnan/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
100%|██████████| 100/100 [34:23<00:00, 20.63s/it]


In [69]:
print(xgb_attack_guesses.shape)
print(xgb_attack_proba.shape)
print(xgb_attack_real.shape)

(200,)
(200, 2)
(200,)


In [70]:
xgb.__version__

'1.4.2'

In [71]:
accuracy = accuracy_score(xgb_attack_real,xgb_attack_guesses)
fpr, tpr, thresholds = metrics.roc_curve(xgb_attack_real, xgb_attack_proba[:,1], pos_label=1)
auc = metrics.auc(fpr, tpr)

print('Privacy of defender model:')
print('(performance of the oracle attack model)')
print('acc: {}'.format(accuracy))
print('auc: {}'.format(auc))

Privacy of defender model:
(performance of the oracle attack model)
acc: 0.91
auc: 0.9756
